In [67]:
def getInput(day):
   f = open("day" + str(day) + "_22" + ".input", "r")
   return f.readlines()

In [68]:
# day 1
import numpy as np

# Splits an array of strings (representing lines of an file in our case)
# into an array of two arrays of strings, the first containing all strings/lines 
# before the first blank line ('\n'), while the last contains those after the
# blank line. The blank line itself is not part of any of these two.
# If the input array of strings contains no blank line at all, this function 
# returns an array containing the inputs array of strings.  
def splitOnFirstBlankLine(stringArray):
    if '\n' in stringArray:
        return [stringArray[:stringArray.index('\n')], stringArray[stringArray.index('\n')+1:]]
    else:
        return [stringArray]

# splits an arrays of strings (representing lines of an file in our case) 
# into an array of arrays of strings, each containing the strings between to
# blank lines ('\n')
# For example the file
# A
# B
#
# C
#
# D
# E
# would be split up into [['A', 'B'], ['C'], ['D', 'E']]
def splitOnBlankLines(stringArray, result=[]):
    splits = splitOnFirstBlankLine(stringArray)
    result.append(splits[0])
    if len(splits) == 1:
        return result
    else: # len == 2
        return splitOnBlankLines(splits[1], result)

# Makes an array of arrays of Integers from an input array of arrays of strings.
# Input array must not contain anything else than an array of arrays of strings,
# otherwise an exception will be thrown
# For example, [['2', '3'], ['5'], ['7', '11']] will be converted into
# [[2, 3], [5], [7, 11]]
def intify(arrayOfStringArrays):
    arrayOfIntArrays = []
    for stringArray in arrayOfStringArrays:
        intArray = [int(string) for string in stringArray]
        arrayOfIntArrays.append(intArray)
    return arrayOfIntArrays

def solveDay1(task):
    # read input
    inputLines = getInput(1)
    
    # split up
    splittedIntoChunks = splitOnBlankLines(inputLines)
    
    # make integers
    intChunks = intify(splittedIntoChunks)
    
    # make an array of sums of chunks
    sums = [np.sum(split) for split in intChunks]
    # and sort by size
    sums.sort()
    
    if task == 1: 
        # the top elve
        return sums[-1]
    if task == 2:
        # three top elves summed together
        return np.sum(sums[-3:])
    
print(solveDay1(1)) 
print(solveDay1(2)) 


72511
214836


In [69]:
#day 2
import numpy as np

# the first half of scoresByShape is used by both tasks (intpretation of A, B, C):
# A for Rock, B for Paper, and C for Scissors.
# the second half of scoresByShape is only needed for the first task (wrong interpretation of X, Y, Z):
# X for Rock, Y for Paper, and Z for Scissors.
scoresByShape = {'A':1, 'B':2, 'C':3, 'X':1, 'Y':2, 'Z':3}

# only for the second task (real interpretation of X, Y, Z)
# X means you need to lose, Y means you need to end the round in a draw, and Z means you need to win.
neededScores = {'X':0, 'Y':3, 'Z':6}

def cantorPairing(x,y):
    return (pow(x,2) + x + 2 * x * y + 3 * y + pow(y,2))/2

# the points that we (with nyShape) receive against them (with theirShape)
def outcomeScore(theirShape, myShape):
    ts = scoresByShape[theirShape]
    ms = scoresByShape[myShape.rstrip()]
    # ugly ternary expression - can replaced with some math involving cantor and interpolation polynomial see below
    # return 3 if ts == ms else (6 if (ms == 1 and ts == 3) else (0 if (ms == 3 and ts == 1) else (0 if ts > ms else 6)))
    x = cantorPairing(ts, ms)
    return int(round(331 * pow(x,8)/304944640 - 921467 * pow(x,7)/6861254400 + 16026271 * pow(x,6)/2287084800 - 63041351 * pow(x,5)/311875200 + 489434749 * pow(x,4)/138611200 - 261604767803 * pow(x,3)/6861254400 + 8799384401 * pow(x,2)/35735700 - 3137692909 * x/3665200 + 421473/350))

# the points we score in a duell, represented by a string defined by 
# '<THEIR_SHAPE><SINGLE_SPACE><THEIR_SHAPE><OPTIONAL_NEWLINE>', 
# where every shape is one of  A, B, C, X, Y, Z. Examples: 
# - 'C Y'
# - 'A Z\n'
# - 'B Z'
# - 'A C'
# - 'X Y\n'
#  or similar
def score(duell):
    shapes = duell.split(' ')
    return outcomeScore(shapes[0], shapes[1]) + scoresByShape[shapes[1].rstrip()]

# the duell given by a strategy, represented by a string defined by 
# '<THEIR_SHAPE><SINGLE_SPACE><STRATEGIC_OUTCOME><OPTIONAL_NEWLINE>',
# where every shape is one of  A, B, C and 
# each strategic outcome one of X, Y, Z. Examples:
# - 'A Y'
# - 'A Z\n'
# - 'B Z'
#  or similar
def duellFor(strategy):
    strategyParts = strategy.split(' ')
    theirShape = strategyParts[0]
    neededScore = neededScores[strategyParts[1].rstrip()]
    for shape in ['A', 'B', 'C']:
        if outcomeScore(theirShape, shape) == neededScore:
            return theirShape + ' ' + shape
    return 'oops' # should not come here

def solveDay2(task):
    # determine duells
    if task == 1:
         duells = getInput(2)
    if task == 2:
        strategies = getInput(2)
        duells = [duellFor(strategy) for strategy in strategies]
    
    # calculate all scores
    scores = [score(duell) for duell in duells]

    # sum up scores and return
    return np.sum(scores)
    
print(solveDay2(1))
print(solveDay2(2))

15691
12989


In [70]:
# day 3
import numpy as np

def priority(itemType):
    corr =  96 if itemType.lower() == itemType else 38
    return ord(itemType) - corr

def inAllCollections(itemType, collections):
    for collection in collections:
        if not itemType in collection:
            return False
    return True

def itemTypeInAllCollections(collections):
    for itemType in collections[0]:
        if inAllCollections(itemType, collections):
            return itemType

def itemTypeInBothCompartments(rucksack):
    compartment1 = rucksack[:int(len(rucksack)/2)]
    compartment2 = rucksack[int(len(rucksack)/2):]
    return itemTypeInAllCollections([compartment1, compartment2])

def solveDay3(task):
    rucksacks = getInput(3)
    if task == 1:
        wrongItemTypes = [itemTypeInBothCompartments(rucksack) for rucksack in rucksacks]
        wrongPriorities = [priority(itemType) for itemType in wrongItemTypes]
        return np.sum(wrongPriorities)
    if task == 2:
        groups = np.array_split(rucksacks, int(len(rucksacks)/3))
        groupItemTypes = [itemTypeInAllCollections(group) for group in groups]
        groupPriorities = [priority(itemType) for itemType in groupItemTypes]
        return np.sum (groupPriorities)
    
print(solveDay3(1))
print(solveDay3(2))

8153
2342


In [71]:
# day 4

def oneIncludesOther(pair):
    both = pair.split(',')
    first = [int(x) for x in both[0].split('-')]
    second= [int(x) for x in both[1].split('-')]
    return first[0] <= second[0] and first[1] >= second[1] or first[0] >= second[0] and first[1] <= second[1]

def overlaps(pair):
    both = pair.split(',')
    first = [int(x) for x in both[0].split('-')]
    second= [int(x) for x in both[1].split('-')]
    return first[1] >= second[0] and first[0] <= second[1] or first[0] <= second[1] and first[1] >= second[0]

def solveDay4(task):
    pairs = getInput(4)
    if task == 1:
        result = 0
        for pair in pairs:
            if oneIncludesOther(pair):
                result += 1
        return result
    if task == 2:
        result = 0
        for pair in pairs:
            if overlaps(pair):
                result += 1
        return result

print(solveDay4(1))
print(solveDay4(2))

599
928


In [72]:
# day 5
def stacksFrom(data):
    rows = []
    for line in data:
        if '[' in line:
            line = line.rstrip()
            rows.append(line.split(' '))
    #transpose
    stacks = [[row[i] for row in rows] for i in range(len(rows[0]))]
    #clean
    for stack in stacks:
        while '[0]'in stack:
            stack.remove('[0]')
    return stacks

def movesFrom(data):
    moves = []
    for line in data:
        if 'move' in line:
            line = line.rstrip()
            moves.append(list(map( lambda s: int(s), list(filter(lambda s: s.isdigit(), line.split(' '))))))
    return moves

def stacksAfterMove(stacks, move, task):
    sourceIdx = move[1]-1
    targetIdx = move[2]-1
    if task == 1:
        stacks[targetIdx] = stacks[sourceIdx][:move[0]][::-1] + stacks[targetIdx]
    else:
        stacks[targetIdx] = stacks[sourceIdx][:move[0]] + stacks[targetIdx]
    stacks[sourceIdx] = stacks[sourceIdx][move[0]:]
    return stacks

def solveDay4(task):
    data = getInput(5)
    stacks = stacksFrom(data)
    moves = movesFrom(data)
    for move in moves:
        stacks = stacksAfterMove(stacks, move, task)
    message = ''
    for stack in stacks:
        message = message + stack[0][1]
    return message

print(solveDay4(1))
print(solveDay4(2))

TLNGFGMFN
FGLQJCMBD


In [73]:
# day 6

def numberProcessedBeforeReachingDistinctOfLenghth(numberOfDistinct):
    data = list(getInput(6)[0])
    for i in range(len(data) - numberOfDistinct):
        chunk = data[i:i+numberOfDistinct]
        if len(set(chunk)) == numberOfDistinct:
            return i + numberOfDistinct


def solveDay6(task):
    if task == 1:
        return numberProcessedBeforeReachingDistinctOfLenghth(4)
    if task == 2:
        return numberProcessedBeforeReachingDistinctOfLenghth(14)

print(solveDay6(1))
print(solveDay6(2))

1658
2260


In [74]:
# day 7

from collections import namedtuple
import numpy as np

File = namedtuple('File', ['name', 'size'])
Dir = namedtuple('Dir', ['name', 'files', 'subdirs'])

def sizeOf(dir):
    size = np.sum([file.size for file in dir.files])
    for subdir in dir.subdirs:
        size = size + sizeOf(subdir)
    return size

def subdirWithName(dirs, name, searchDeep=False):
    for dir in dirs:
        for subdir in dir.subdirs:
            if subdir.name == name:
                return subdir
    if searchDeep:
        for dir in dirs:
            return subdirWithName(dir.subdirs, name)

def hasFileWithName(name, dir):
    for file in dir.files:
        if file.name == name:
            return True
    return False


def allDirSizes(dir, sizes=[]):
    sizes.append(sizeOf(dir)) 
    for subdir in dir.subdirs:
        allDirSizes(subdir, sizes)
    return sizes

def buildDirTree(harddisk):
    root = Dir('/', [], [])
    currentPath = []
    for line in harddisk:
        if line[:6] == '$ cd /':
            currentPath = [root]
        elif line[:7] == '$ cd ..':
            currentPath = currentPath[:len(currentPath)-1]
        elif line[:4] == '$ cd':
            name = line[5:].rstrip()
            currentDir = currentPath[-1]
            stepIntoDir = subdirWithName([currentDir], name)
            if stepIntoDir is None:
                stepIntoDir = Dir(name, [], [])
                currentDir.subdirs.append(stepIntoDir)
            currentPath.append(stepIntoDir)
        elif line[0].isnumeric():
            size = int(line.split(' ')[0])
            name = (line.split(' ')[1]).rstrip()
            currentDir = currentPath[-1]
            if not hasFileWithName(name, currentDir):
                newFile = File(name, size)
                currentDir.files.append(newFile) 
    return root        


def solveDay7(task):
    root = buildDirTree(getInput(7))
    allSizes = allDirSizes(root)
    if task == 1:
        return np.sum(list(filter(lambda size: size <= 100000, allSizes)))
    if task == 2:
        availableSpace = 70000000 - sizeOf(root)
        toDelete = 30000000 - availableSpace
        possibleSizes = list(filter(lambda size: size >= toDelete, allSizes))
        possibleSizes.sort()
        return possibleSizes[0]

print(solveDay7(1))
print(solveDay7(2))

1348005.0
12785886.0


In [75]:
# day 8

TreeEnsemble = namedtuple('Dir', ['tree', 'leftRow', 'topColumn', 'rightRow', 'bottomColumn'])

def treeEnsembleAt(position, forrest):
    tree = forrest[position[0]][position[1]]
    treesRow = (forrest[position[0]])
    treesColumn = [forrest[i][position[1]] for i in range(len(forrest))]
    return TreeEnsemble(tree, treesRow[:position[1]], treesColumn[:position[0]], treesRow[position[1]+1:], treesColumn[position[0]+1:])

def scenicScore(treePosition, theForest):
    treeEnsemble = treeEnsembleAt(treePosition, theForest)
    leftScore = 0
    for t in reversed(treeEnsemble.leftRow):
        leftScore = leftScore + 1
        if t >= treeEnsemble.tree:
            break
    rightScore = 0
    for t in treeEnsemble.rightRow:
        rightScore = rightScore + 1
        if t >= treeEnsemble.tree:
            break
    topScore = 0
    for t in reversed(treeEnsemble.topColumn):
        topScore = topScore + 1
        if t >= treeEnsemble.tree:
            break 
    bottomScore = 0
    for t in treeEnsemble.bottomColumn:
        bottomScore = bottomScore + 1
        if t >= treeEnsemble.tree:
            break 
    return leftScore * rightScore * topScore * bottomScore

def isVisible(treePosition, theForest):
    treeEnsemble = treeEnsembleAt(treePosition, theForest)
    if (len(list(filter(lambda aTree: aTree >= treeEnsemble.tree, treeEnsemble.leftRow))) == 0):
        return True
    if (len(list(filter(lambda aTree: aTree >= treeEnsemble.tree, treeEnsemble.rightRow))) == 0):
        return True
    if (len(list(filter(lambda aTree: aTree >= treeEnsemble.tree, treeEnsemble.topColumn))) == 0):
        return True
    if (len(list(filter(lambda aTree: aTree >= treeEnsemble.tree, treeEnsemble.bottomColumn))) == 0):
        return True
    return False

def solveDay8(task):
    input = [list(line)[:-1] for line in getInput(8)]
    theForest = [[int(n) for n in line] for line in input]
    height = len(theForest)
    width = len(theForest[0])
    if task == 1:
        visibleTrees = []
        for i in range(1, height-1):
            for j in range(1, width-1):
                if isVisible([i,j], theForest):
                    visibleTrees.append(theForest[i][j])
        return len(visibleTrees) + 2 * width + 2 * (height-2)
    if task == 2:
        highScore = 0
        for i in range(1, height-1):
            for j in range(1, width-1):
                newScore = scenicScore([i,j], theForest)
                if newScore > highScore:
                    highScore = newScore
        return highScore

print(solveDay8(1))
print(solveDay8(2))

1676
313200


In [76]:
# day 9

import numpy as np

rightDirection = np.array([1,0])
leftDirection = np.array([-1,0])
upDirection = np.array([0,1])
downDirection = np.array([0,-1])
directions = {'R': rightDirection, 'L': leftDirection, 'U': upDirection, 'D': downDirection}

def drawKnots(rope):
    drawRope = [[rope[ri][0], rope[ri][1]] for ri in range(len(rope))]
    for y in range(15,-16, -1):
        line = []
        for x in range(-15, 16):
            if [x,y] in drawRope:
                idx = drawRope.index([x,y])
                if idx == 0:
                    line.append('H')
                elif idx == 9:
                    line.append('T')
                else:
                    line.append(str(idx))
            else:
                if x == 0 and y == 0:
                    line.append('s')
                else:
                    line.append('.')
        print(''.join(line) + str(y))
    
def positionsTouching(pos1, pos2):
    distance = np.abs(np.subtract(pos1, pos2))
    return distance[0] <= 1 and distance[1] <=1

def newKnotPosition(prevKnotPos, oldKnotPos, move):
    if positionsTouching(prevKnotPos, oldKnotPos):
        return oldKnotPos
    elif prevKnotPos[0] == oldKnotPos[0]: # same xPos
        if prevKnotPos[1] > oldKnotPos[1]:
            return np.array([prevKnotPos[0], prevKnotPos[1]-1])
        else:
            return np.array([prevKnotPos[0], prevKnotPos[1]+1])
    elif prevKnotPos[1] == oldKnotPos[1]: # same yPos
        if prevKnotPos[0] > oldKnotPos[0]:
            return np.array([prevKnotPos[0]-1, prevKnotPos[1]])
        else:
            return np.array([prevKnotPos[0]+1, prevKnotPos[1]])
    else: # diagonal
        if move[0] == 0: # up or down 
            if prevKnotPos[0] > oldKnotPos[0]: # to the right
                newX = oldKnotPos[0] + 1
            else: # to the left
                newX = oldKnotPos[0] - 1    
            if move[1] > 0: # up
                newY = oldKnotPos[1] + 1
            else: # down
                newY = oldKnotPos[1] -1
        else: # right or left
            if prevKnotPos[1] > oldKnotPos[1]: # up
                newY = oldKnotPos[1] + 1
            else: # down
                newY = oldKnotPos[1] - 1    
            if move[0] > 0: # left
                newX = oldKnotPos[0] + 1
            else: # right
                newX = oldKnotPos[0] -1
        return np.array([newX, newY])

def movesFromBigMove(bigMove):
    dirCommand = bigMove.split(' ')[0]
    stepCount = int((bigMove.split(' ')[1]).rstrip())
    return [directions[dirCommand]] * stepCount 

def solveDay9(task):
    bigMoves = getInput(9)
    if task == 1:
        knotCount = 2
    if task == 2:
        knotCount = 10
    currentRope = [np.array([0,0])] * knotCount
    tailPositions = [tuple([0, 0])]
    for bigMove in bigMoves:
        moves = movesFromBigMove(bigMove)
        for move in moves:
            currentRope[0] = np.add(currentRope[0], move)
            newMove = move
            for i in range(1, knotCount):
                newCurrentI = newKnotPosition(currentRope[i-1], currentRope[i], newMove)
                newMove = np.subtract(newCurrentI, currentRope[i])
                currentRope[i] = newCurrentI
            tailPositions.append(tuple([currentRope[knotCount-1][0], currentRope[knotCount-1][1]]))
        # drawKnots(currentRope)
    return len(set(tailPositions))


print(solveDay9(1))
print(solveDay9(2))

6314
2504


In [109]:
# day 10

def drawNext(x, cycle, line, drawing):
    next = '#' if ((x-1) == (cycle - line) or x == (cycle - line) or (x+1) == (cycle - line)) else ' '
    return drawing + next

def checkNewLine(cycle, line, drawing):
    if (cycle % 40) == 0:
        drawing = drawing + '\n'
        line = line + 40
    return (line, drawing)

def screenFor(processSteps):
    x = 1
    line = 0
    cycles = 0
    drawing = ''
    for step in processSteps:
        if len(step) == 1:
            drawing = drawNext(x, cycles, line, drawing)
            cycles += 1
            line, drawing = checkNewLine(cycles, line, drawing)
        elif len(step) == 2:
            drawing = drawNext(x, cycles, line, drawing)
            cycles += 1
            line, drawing = checkNewLine(cycles, line, drawing)
            drawing = drawNext(x, cycles, line, drawing)
            cycles += 1
            line, drawing = checkNewLine(cycles, line, drawing)
            x += int(step[1].rstrip())
    return drawing
        

def registerValueDuringCycle(n, processSteps):
    x = 1
    cycles = 1
    for step in processSteps:
        if len(step) == 1:
            cycles += 1
            if cycles == n:
                break
        elif len(step) == 2:
            if cycles >= (n-1):
                break
            cycles += 2
            x += int(step[1].rstrip())
    return x

def solveDay10(task):
    processSteps = [step.split(' ') for step in getInput(10)]
    signalStrengthSum = 0
    if task == 1:
        for n in range(20, 221, 40):
            signalStrength = n * registerValueDuringCycle(n, processSteps)
            signalStrengthSum += signalStrength
        return signalStrengthSum
    if task == 2:
        return screenFor(processSteps)

print(solveDay10(1))
print(solveDay10(2))

14160
###    ## #### ###  ###  #### ####  ##  
#  #    # #    #  # #  # #    #    #  # 
#  #    # ###  #  # #  # ###  ###  #    
###     # #    ###  ###  #    #    #    
# #  #  # #    # #  #    #    #    #  # 
#  #  ##  #### #  # #    #### #     ##  

